In [83]:
import os
import rasterio
import numpy as np
import geopandas as gpd

from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon, box


class SerialTimer():
    
    def __init__(self,
                 aoi=[15.00, 48.00, 15.01, 48.01],
                 raster_dir='/home/volume2/Austria/Timeseries/',
                 polygon_file='/home/volume2/Austria/GroundTruth/invekos_schlaege_polygon.shp',
                 out_dir='/home/mlamarre/Documents/FieldTimeSeries',
                 time_stamps=np.arange(1,61),
                 dimensions=['BS.VH', 'BS.VV']):
        
        self.raster_dir = raster_dir
        self.polygon_file = polygon_file
        self.out_dir = out_dir
        self.aoi = box(minx=aoi[0],miny=aoi[1],maxx=aoi[2],maxy=aoi[3])
        self.time_stamps = time_stamps
        self.dimensions = dimensions
        self.scanned = False
        
    def readPolygons(self):
        if os.path.isfile(self.polygon_file):
            print('Reading shape file...')
            self.polygons = gpd.read_file(self.polygon_file)
            self.poly_crs = self.polygons.crs
        else:
            print('ERROR: wrong shape file')
            
    def readRaster(self, time_stamp, dimension):
        filename = '.'.join([self.raster_dir+str(time_stamp),dimension,'tif'])
        if os.path.isfile(filename):
            print('Reading raster file...')
            self.raster = rasterio.open(filename)
            self.rast_crs = self.raster.crs
            # Get the values of raster as numpy array and the transform of the raster
            self.array = self.raster.read(1)
            self.transform = self.raster.transform
        
        else:
            print('ERROR: wrong raster file')
            
    def selectPolygons(self):
        # Transforms AoI to GeoDataFrame to change CRS and then back to Polygon to use within function
        print('Original coordinates: {}'.format(self.aoi.bounds))
        AOI = gpd.GeoDataFrame({'geometry':self.aoi}, index=[0], crs=self.rast_crs)
        AOI = AOI.to_crs(self.poly_crs)
        AOI = box(minx=AOI.bounds.loc[0]['minx'],
                  miny=AOI.bounds.loc[0]['miny'],
                  maxx=AOI.bounds.loc[0]['maxx'],
                  maxy=AOI.bounds.loc[0]['maxy'])
        print('Transformed coordinates: {}'.format(AOI.bounds))
        
        # Iterate over all the polygons and check if the 1st point is within AOI 
        idxs = []
        for index, row in self.polygons.iterrows():
            if index % 2500 == 0:
                print('Scanned {} out of {} fields. ({}%)'.format(index,self.polygons.shape[0],index/self.polygons.shape[0]*100),end='\r')
            if(Point(row.geometry.exterior.coords[0]).within(AOI)):
                idxs.append(index)        
        print('{} fields were selected'.format(len(idxs)))
        
        # Keep the polygons of interest and change their CRS
        self.polys_OI = self.polygons.iloc[idxs]
        self.polys_OI = self.polys_OI.to_crs(self.rast_crs)
        
        # Drop irrelevant columns and reset indices
        self.polys_OI = self.polys_OI.drop(columns = ['FS_KENNUNG', 'SL_FLAECHE'])
        self.polys_OI = self.polys_OI.reset_index(drop=True)
        
    def getZonalStats(self, time_stamp, dimension):
        # Selected polygons and get the zonal stats
        polys_stats = zonal_stats(self.polys_OI['geometry'], self.array, affine=self.transform, stats=['mean'])
        polys_stats = [v['mean'] for v in polys_stats]
        self.polys_OI[dimension + str(time_stamp)] = polys_stats
        
    def mergeColums(self, dimension):
        # Merge columns of same time series
        columns_series = [dimension + str(t) for t in self.time_stamps]
        self.polys_OI[dimension] = self.polys_OI[columns_series].values.tolist()
        # Drop indivuals columns
        self.polys_OI = self.polys_OI.drop(columns = columns_series)
                
    def routine(self):
        
        # Read all the polygons
        self.readPolygons()
        
        #Iterate over dimensions and time
        for d in self.dimensions:
            for t in self.time_stamps:
                
                # Read each dimension- and time-specific raster
                self.readRaster(time_stamp=t, dimension=d)
                
                # Select the polygons only once
                if not self.scanned:
                    self.selectPolygons()
                    self.scanned = True
                    
                # Get the information for all the polygons
                self.getZonalStats(time_stamp=t, dimension=d)
                
            # Merge the columns into a time series
            self.mergeColumns(dimension=d)
            
                

In [84]:
s = SerialTimer(time_stamps=[1,2], dimensions=['BS.VV','BS.VH'])

In [85]:
s.routine()

Reading shape file...
Reading raster file...
Original coordinates: (15.0, 48.0, 15.01, 48.01)
Transformed coordinates: (524362.7234763639, 456961.1201626792, 525132.3808825933, 458088.4550425114)
46 fields were selected2522622 fields. (99.99516376214906%))


/home/mlamarre/miniconda3/lib/python3.7/site-packages/rasterstats/io.py:300: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


Reading raster file...


AttributeError: 'SerialTimer' object has no attribute 'mergeColumns'

In [3]:
s = SerialTimer()

In [4]:
s.readPolygons()

Reading shape file...


In [5]:
s.readRaster(1,'BS.VV')

Reading raster file...


In [6]:
s.selectPolygons()

Original coordinates: (15.0, 48.0, 15.01, 48.01)
Transformed coordinates: (524362.7234763639, 456961.1201626792, 525132.3808825933, 458088.4550425114)
46 fields were selected2522622 fields. (99.99516376214906%))


In [17]:
bigby = s.polys_OI


In [18]:
bigby = bigby.drop(columns = ['FS_KENNUNG', 'SL_FLAECHE'])
bigby = bigby.reset_index(drop=True)

In [53]:
# Get the values of raster as numpy array and the transform of the raster
array = s.raster.read(1)
transform = s.raster.transform
        
# Iterate over the selected polygons and get the zonal stats
polys_stats = []
polys_stats = zonal_stats(bigby['geometry'], array, affine=transform, stats=['mean'])
polys_stats = [v['mean'] for v in polys_stats]
    
bigby['BS.VV2'] = polys_stats

In [74]:
bigby['timy'] = bigby[['BS.VV','BS.VV2']].values.tolist()
bigby

,SNAR_BEZEI,ID,geometry,BS.VV,BS.VV2,timy
0,DAUERWEIDE,171899,"POLYGON ((15.00084 48.00866, 15.00075 48.00868...",-10.685688,-10.685688,"[-10.68568766117096, -10.68568766117096]"
1,DAUERWEIDE,171900,"POLYGON ((15.00148 48.00910, 15.00149 48.00911...",-15.029026,-15.029026,"[-15.029025713602701, -15.029025713602701]"
2,MÄHWIESE/-WEIDE DREI UND MEHR NUTZUNGEN,171901,"POLYGON ((15.00058 48.00690, 15.00064 48.00693...",-13.593633,-13.593633,"[-13.5936332543691, -13.5936332543691]"
3,MÄHWIESE/-WEIDE DREI UND MEHR NUTZUNGEN,171902,"POLYGON ((15.00718 48.00779, 15.00719 48.00780...",-13.224385,-13.224385,"[-13.224385397774833, -13.224385397774833]"
4,MÄHWIESE/-WEIDE DREI UND MEHR NUTZUNGEN,171903,"POLYGON ((15.00048 48.00724, 15.00050 48.00740...",-12.098636,-12.098636,"[-12.09863612651825, -12.09863612651825]"
5,DAUERWEIDE,171904,"POLYGON ((15.00046 48.00907, 15.00030 48.00902...",-13.541568,-13.541568,"[-13.541568100452423, -13.541568100452423]"
6,MÄHWIESE/-WEIDE DREI UND MEHR NUTZUNGEN,171906,"POLYGON ((15.00144 48.00923, 15.00153 48.00923...",-12.956392,-12.956392,"[-12.956392434335523, -12.956392434335523]"
7,MÄHWIESE/-WEIDE DREI UND MEHR NUTZUNGEN,171907,"POLYGON ((15.00719 48.00785, 15.00693 48.00785...",-12.639624,-12.639624,"[-12.639624495255319, -12.639624495255319]"
8,LSE HECKE / UFERGEHÖLZ,171908,"POLYGON ((15.00284 48.00882, 15.00304 48.00888...",-15.049969,-15.049969,"[-15.049968719482422, -15.049968719482422]"
9,LSE HECKE / UFERGEHÖLZ,171918,"POLYGON ((15.00445 48.00838, 15.00443 48.00839...",-9.482411,-9.482411,"[-9.48241138458252, -9.48241138458252]"
